# Remapping: Analysis of pairs of neurons during open field foraging, light trials and dark trials

We get the IFR and the firing rate maps for the neurons in different conditions. We then look at the IFR association and firing rate map similarity for pairs of neurons in a given condition.

* Do cells that fire together in one environment also fire together in the next environment?
* Do cells with similar firing rate maps in one condition also have similar maps in another?


In [1]:
%load_ext autoreload
%autoreload 2

%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/setup_project.py
%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/neuronAutopi.py

prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1
dlcModelPath: /adata/models
Reading /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:14<00:00,  2.77it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [2]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 48.18it/s]


In [5]:
myProject.dataPath

'/ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1'

## Instantaneous Firing rate associations

We can look at pairs of cells and test if their firing associations (r values) are preserve or change across different conditions.
The function below calculates the firing associations of pairs of neurons in different conditions.
Later on we can compare IFR association across conditions.

In [6]:
from scipy.stats import pearsonr
def ifrAssociationsPerCondition(ses,sSes,conditions=["circ80","circ80_1","circ80_2","all_task","all_task_1","all_task_2","light","dark","light_1","light_2","dark_1","dark_2","all_light","all_dark","all_light_1","all_light_2","all_dark_1","all_dark_2"],prefix="ifrAsso_"):
    """
    Calculate the instantaneous firing rate associations of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with the IFR association for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    for cond in conditions:
        # set the intervals for each neuron and calculate the IFR
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(ses.intervalDict[cond])
            n.spike_train.instantaneous_firing_rate(bin_size_sec = 0.100, sigma = 1,outside_interval_solution="remove")
        # calculate firing association for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = pearsonr(n1.spike_train.ifr[0].flatten(),n2.spike_train.ifr[0].flatten())[0]
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

def spikeTimeCrosscorrelation(ses,sSes):
    """
    To make sure the pairs do not have common refractory periods
    """
    for n in sSes.cg.neuron_list:
        n.spike_train.unset_intervals()
        
    sSes.cg.make_pairs(pair_type="combinations")
    
    # get the size of one crosscorrelation
    st0 = sSes.cg.neuron_list[0].spike_train
    st1 = sSes.cg.neuron_list[1].spike_train
    myHist,myRange = st0.spike_time_crosscorrelation(st1=None,st2=st1,bin_size_sec=0.0005, min_sec=-0.05, max_sec=0.05)
    res = np.empty((len(sSes.cg.pairs),myHist.shape[0]))
    for i, (j,k) in enumerate(sSes.cg.pairs):
        n1=sSes.cg.neuron_list[j]
        n2=sSes.cg.neuron_list[k]
        myHist,myRange = n1.spike_train.spike_time_crosscorrelation(st1=None,st2=n2.spike_train, bin_size_sec=0.0005, min_sec=-0.05, max_sec=0.05)
        res[i,:]=myHist
    return res


Run on one session

In [7]:
ses.intervalDict.keys()

dict_keys(['circ80', 'circ80_1', 'circ80_2', 'task', 'task_1', 'task_2', 'light', 'light_1', 'light_2', 'dark', 'dark_1', 'dark_2', 'searchPath_light', 'searchPath_light_1', 'searchPath_light_2', 'searchPath_dark', 'searchPath_dark_1', 'searchPath_dark_2', 'searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingPath_light', 'homingPath_light_1', 'homingPath_light_2', 'homingPath_dark', 'homingPath_dark_1', 'homingPath_dark_2', 'homingFromLeavingLever_light', 'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2', 'homingFromLeavingLeverToPeriphery_light', 'homingFromLeavingLeverToPeriphery_light_1', 'homingFromLeavingLeverToPeriphery_light_2', 'homingFromLeavingLeverToPeriphery_dark', 'homingFromLeavingLeverToPeriphery_dark_1', 'homingFromLeavingLeverToPeriphery_

In [8]:
%%time
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = ifrAssociationsPerCondition(ses,sSes)
res = spikeTimeCrosscorrelation(ses,sSes)

/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


CPU times: user 5.99 s, sys: 2.38 s, total: 8.37 s
Wall time: 5.67 s


In [9]:
len(myProject.sessionList)

39

Run on all sessions

In [10]:
import warnings
from scipy.stats import ConstantInputWarning

# Suppress the ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.resetwarnings()


In [11]:
res_ifr=pd.concat([ifrAssociationsPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

0it [00:00, ?it/s]/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
3it [00:17,  5.33s/it]/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
7it [00:36,  5.26s/it]/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
8it [00:45,  6.40s/it]/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
 

In [12]:
res_cc=np.concatenate([spikeTimeCrosscorrelation(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))])

39it [00:22,  1.73it/s]


## Firing rate map similarities

As a complementary analysis to the IFR association, we can also look at firing rate map similarity in different conditions. 


In [13]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    r,p = pearsonr(a[indices],b[indices])
    return r

def mapSimilarityPerCondition(ses,sSes,
                              conditions=["circ80","circ80_1","circ80_2","all_task","all_task_1","all_task_2","light","dark","light_1","light_2","dark_1","dark_2","all_light","all_dark","all_light_1","all_light_2","all_dark_1","all_dark_2"],
                                          prefix="mapSim_"):
    """
    Calculate the map similarity of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with map similarity for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    xy_range=np.array([[-50,-90],[50,60]])
    
    for cond in conditions:
        # set the intervals for each neuron and animal pose and calculate the maps
        
        sSes.ap.set_intervals(ses.intervalDict[cond])
        
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(ses.intervalDict[cond])
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
            
            
        # calculate map similarity for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = map_cor(n1.spatial_properties.firing_rate_map,n2.spatial_properties.firing_rate_map)
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

In [14]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = mapSimilarityPerCondition(ses,sSes)
res

/home/maryam/repo/spikeA/spikeA/Animal_pose.py:557: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  occ_sm = ndimage.filters.gaussian_filter(occ,sigma=smoothing_sigma_cm/cm_per_bin)
/home/maryam/repo/spikeA/spikeA/Spatial_properties.py:428: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  spike_count = ndimage.filters.gaussian_filter(spike_count,
/home/maryam/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


,mapSim_circ80,mapSim_circ80_1,mapSim_circ80_2,mapSim_all_task,mapSim_all_task_1,mapSim_all_task_2,mapSim_light,mapSim_dark,mapSim_light_1,mapSim_light_2,...,mapSim_dark_2,mapSim_all_light,mapSim_all_dark,mapSim_all_light_1,mapSim_all_light_2,mapSim_all_dark_1,mapSim_all_dark_2,id1,id2,session
0,0.188970,0.377550,-0.063779,0.205488,0.343285,0.056575,0.298867,0.340701,-0.008510,0.403513,...,0.369308,0.279496,0.364421,0.268825,0.318663,0.231388,0.464270,mn5824-20112020-0107_80,mn5824-20112020-0107_90,mn5824-20112020-0107
1,0.397997,0.078335,0.539016,0.412078,0.171506,0.330584,0.015716,-0.260255,0.112108,0.002370,...,-0.145423,0.029916,-0.006507,-0.002201,0.152785,-0.003764,-0.069572,mn5824-20112020-0107_80,mn5824-20112020-0107_92,mn5824-20112020-0107
2,-0.331926,NaN,-0.326266,-0.057822,-0.020545,-0.371787,-0.149790,0.030228,0.222013,-0.290168,...,0.135246,-0.440025,0.385069,-0.283628,-0.211709,0.056279,0.424262,mn5824-20112020-0107_80,mn5824-20112020-0107_96,mn5824-20112020-0107
3,-0.372722,-0.121031,-0.524874,0.034142,0.062580,-0.027156,0.045847,0.404098,-0.241365,-0.020474,...,0.163029,-0.178536,0.165811,-0.153011,-0.197872,0.138247,0.187844,mn5824-20112020-0107_80,mn5824-20112020-0107_98,mn5824-20112020-0107
4,-0.114121,0.195360,-0.348479,-0.188524,-0.197088,-0.011370,-0.062324,-0.132594,0.160796,-0.280458,...,-0.088105,0.209642,-0.223040,0.509009,-0.222877,-0.036961,-0.361798,mn5824-20112020-0107_80,mn5824-20112020-0107_100,mn5824-20112020-0107
5,0.524536,0.320344,0.648466,-0.170458,-0.230146,-0.069596,0.147273,0.529993,-0.111778,-0.278229,...,0.348575,0.392981,-0.205038,0.136740,0.056837,-0.241965,-0.091440,mn5824-20112020-0107_80,mn5824-20112020-0107_114,mn5824-20112020-0107
6,0.085550,-0.169098,0.313527,0.347040,0.302062,0.176767,-0.118168,-0.073669,-0.047458,0.052698,...,-0.008602,-0.307832,-0.010282,0.038142,-0.404484,0.234613,-0.107028,mn5824-20112020-0107_80,mn5824-20112020-0107_116,mn5824-20112020-0107
7,-0.069185,0.117498,-0.184838,-0.058966,0.057071,-0.047071,-0.031517,0.341300,-0.105433,0.128759,...,0.058461,-0.374248,-0.020190,0.074900,-0.486876,0.057377,0.201954,mn5824-20112020-0107_80,mn5824-20112020-0107_118,mn5824-20112020-0107
8,-0.170214,-0.036205,-0.320098,0.374679,0.243753,0.349519,0.390012,0.444459,0.097814,0.353948,...,0.162890,0.293319,0.183158,0.124714,0.407020,0.172920,0.154396,mn5824-20112020-0107_80,mn5824-20112020-0107_122,mn5824-20112020-0107
9,NaN,NaN,NaN,0.059393,0.056289,-0.038275,0.064468,0.025812,0.224878,-0.068741,...,0.105533,-0.152478,0.289936,0.111109,-0.210885,0.036917,0.239421,mn5824-20112020-0107_80,mn5824-20112020-0107_124,mn5824-20112020-0107


In [15]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=stats.ConstantInputWarning)


In [16]:
res_map=pd.concat([mapSimilarityPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [09:38, 14.83s/it]


## Check data integrity of ifr and mapSim df and merge

In [17]:
print(res_map.shape,res_ifr.shape)
if res_map.shape != res_ifr.shape:
    print("problem with the shape of the results")
else:
    print("shape of 2 dfs is matching")
    
if res_map.shape[0] != res_cc.shape[0]:
    print("problem with res_map and res_cc")

(22865, 21) (22865, 21)
shape of 2 dfs is matching


In [18]:
res_map.keys()

Index(['mapSim_circ80', 'mapSim_circ80_1', 'mapSim_circ80_2',
       'mapSim_all_task', 'mapSim_all_task_1', 'mapSim_all_task_2',
       'mapSim_light', 'mapSim_dark', 'mapSim_light_1', 'mapSim_light_2',
       'mapSim_dark_1', 'mapSim_dark_2', 'mapSim_all_light', 'mapSim_all_dark',
       'mapSim_all_light_1', 'mapSim_all_light_2', 'mapSim_all_dark_1',
       'mapSim_all_dark_2', 'id1', 'id2', 'session'],
      dtype='object')

In [18]:
res_map.iloc[:,:-3]

,mapSim_circ80,mapSim_circ80_1,mapSim_circ80_2,mapSim_all_task,mapSim_all_task_1,mapSim_all_task_2,mapSim_light,mapSim_dark,mapSim_light_1,mapSim_light_2,mapSim_dark_1,mapSim_dark_2,mapSim_all_light,mapSim_all_dark,mapSim_all_light_1,mapSim_all_light_2,mapSim_all_dark_1,mapSim_all_dark_2
0,0.188970,0.377550,-0.063779,0.205488,0.180371,0.181908,0.298867,0.340701,0.316399,0.342703,0.293538,0.366397,0.279496,0.364421,0.251539,0.190284,0.237440,0.239705
1,0.397997,0.078335,0.539016,0.412078,0.465400,0.291808,0.015716,-0.260255,0.017099,0.096695,-0.207852,-0.295748,0.029916,-0.006507,-0.007716,0.316671,-0.055046,-0.043258
2,-0.331926,NaN,-0.326266,-0.057822,-0.021654,-0.160602,-0.149790,0.030228,-0.415409,0.069096,-0.039391,0.012536,-0.440025,0.385069,-0.394371,-0.115608,0.038383,0.429906
3,-0.372722,-0.121031,-0.524874,0.034142,0.071282,0.043826,0.045847,0.404098,-0.257987,0.235862,0.304437,0.250151,-0.178536,0.165811,-0.108057,-0.320185,0.235779,0.008771
4,-0.114121,0.195360,-0.348479,-0.188524,-0.234500,-0.173529,-0.062324,-0.132594,0.511465,-0.245710,-0.109066,-0.122160,0.209642,-0.223040,0.440230,-0.274978,-0.293093,-0.183915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22860,0.105399,0.190576,0.000304,-0.388186,-0.198327,-0.449674,-0.083839,-0.087382,-0.170654,-0.079388,-0.020768,-0.264386,-0.227147,-0.280473,-0.370764,-0.102151,-0.286604,-0.222405
22861,0.020007,0.051041,0.009694,0.572778,0.623403,0.456452,0.149185,0.825211,0.052370,0.172966,0.799486,0.548624,0.107438,0.798305,0.098089,0.093870,0.780959,0.725458
22862,0.051649,0.108651,-0.078257,0.462997,0.351743,0.437983,0.079716,0.196696,0.292531,-0.008127,0.293835,0.110309,0.298188,0.049384,0.347067,0.332364,0.331382,-0.103564
22863,0.171690,0.156684,0.173986,-0.438736,-0.412911,-0.483591,-0.086173,-0.081663,-0.169081,-0.038713,-0.104941,-0.070142,0.284415,-0.404881,0.306286,0.309919,-0.435804,-0.205117


# Save the data

In [20]:
#res = pd.concat([res_ifr,res_map.iloc[:,0:-3]],axis=1)
res = pd.concat([res_ifr,res_map],axis=1)
fn=myProject.dataPath+"/results/pairs_ifrAsso_mapSim.csv"
print("Saving to",fn)
res.to_csv(fn,index=False)

Saving to /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/pairs_ifrAsso_mapSim.csv


In [21]:
fn=myProject.dataPath+"/results/pairs_cc.npy"
np.save(fn,res_cc)

In [22]:
print('we are done with this file!')

we are done with this file!
